In [3]:
import pandas as pd
import sklearn
import numpy as np

data = pd.read_excel('../Adatok/tisztitott_adat.xlsx', header=0,index_col=0)
data

,date,text,president,label
0,2012-11-05,president obama tells the story of fired up re...,Obama,0
1,2012-11-06,election day is here confirm your polling plac...,Obama,0
2,2012-11-07,thank you president obama in his victory speec...,Obama,-1
3,2012-11-08,the definition of hope is you still believe ev...,Obama,-1
4,2012-11-09,what bobby kennedy called the ripples of hope ...,Obama,0
...,...,...,...,...
1823,2020-06-11,our great national guard troops who took care ...,Trump,-1
1824,2020-06-12,people have no idea how fake the lamestream me...,Trump,-1
1825,2020-06-15,i’ve done more in less than 4 years than biden...,Trump,1
1826,2020-06-16,wow may retail sales show biggest one-month in...,Trump,0


In [11]:
import gensim
tokenized_tweet = data['text'].apply(lambda x: x.split())

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=50, 
            window=5, 
            min_count=2,                                   
            sg = 1,
            hs = 0,
            negative = 10, 
            workers= 32, 
            seed = 34
)

model_w2v.train(tokenized_tweet, total_examples= len(data['text']), epochs=20)

(4996349, 6534600)

In [12]:
model_w2v.wv.most_similar(positive="trump")

[('donald', 0.8813544511795044),
 ('j', 0.7670353651046753),
 ('sabotage', 0.7368723154067993),
 ('frame', 0.685495913028717),
 ('trump’s', 0.678617000579834),
 ('foundation', 0.6685938835144043),
 ('takedown', 0.6636621356010437),
 ('plot', 0.6620488166809082),
 ('clinton', 0.6554353833198547),
 ('campaign', 0.652974545955658)]

In [13]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [14]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 50)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 50)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(1828, 50)

In [15]:
vec_data = pd.concat([wordvec_df, data['label']], axis=1)
vec_data

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,label
0,0.211237,0.511313,0.020558,-0.130269,0.412363,-0.173577,-0.257338,-0.076809,-0.226040,-0.238499,...,-0.495704,0.215028,-0.025649,0.002599,-0.141979,0.266143,-0.360443,-0.102130,-0.254246,0
1,0.262321,0.376918,-0.034491,-0.048589,0.408996,-0.078180,-0.207245,-0.173951,-0.330650,-0.089396,...,-0.628082,0.066327,0.050675,-0.025971,-0.237459,0.233096,-0.577206,-0.005746,-0.302730,0
2,0.152753,0.287909,-0.009970,0.064506,0.382174,-0.133902,-0.183040,-0.086652,-0.282310,-0.067722,...,-0.547999,0.076656,0.099634,-0.085858,-0.172792,0.182504,-0.504729,-0.033667,-0.246986,-1
3,0.227675,0.380395,-0.072408,-0.015062,0.385880,-0.161939,-0.154601,-0.138655,-0.209884,0.005553,...,-0.573807,-0.048522,0.115793,0.028717,-0.117772,0.163182,-0.463737,0.089421,-0.385457,-1
4,0.073455,0.378790,-0.124944,0.005942,0.456297,-0.183703,-0.186649,-0.081883,-0.223629,-0.137716,...,-0.587914,-0.095553,0.160431,-0.090187,-0.168324,0.257795,-0.590201,-0.000375,-0.373713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,0.166577,0.251380,-0.186115,-0.059232,0.246968,-0.030453,-0.191882,-0.095117,-0.324323,0.017951,...,-0.568921,-0.097746,0.196562,-0.041305,-0.194173,0.195119,-0.572311,-0.118294,-0.318756,-1
1824,0.200662,0.301914,-0.110469,-0.019978,0.225036,-0.146430,-0.181285,-0.159599,-0.255236,-0.038092,...,-0.605691,-0.084217,0.161910,-0.016286,-0.156041,0.214333,-0.521417,-0.054399,-0.279514,-1
1825,0.190328,0.335932,-0.265849,-0.018815,0.130668,-0.204347,-0.192384,-0.190700,-0.292305,0.009898,...,-0.580780,-0.146579,0.154963,-0.057078,-0.148703,0.213551,-0.481174,0.023114,-0.254860,1
1826,0.147867,0.225133,-0.196530,-0.110264,0.291163,-0.043916,-0.216164,-0.102982,-0.336679,0.075210,...,-0.520532,-0.126400,0.191995,0.017389,-0.161118,0.190478,-0.570801,-0.136490,-0.325469,0


In [17]:
from sklearn.model_selection import train_test_split

label = vec_data['label']
attrs = vec_data.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(
        attrs, label, test_size=0.25, random_state=1000)

In [19]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.7242888402625821


In [20]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30, random_state=0)
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.7177242888402626
